**
We want to build such a classifier, which will give an opportunity for each student to estimate the probability of him answering a certain question correctly, taking into account the data of that student and the complexity of the question.

Student data will be used by him
* gender,
* average time to answer previous questions (avg_time),
* average proportion of correct answers to previous questions (avg_correct)

As a description of the complexity of the question, we will use

* time allocated to the question (total_time)
* the number of words in the content of the question (question)
* the sum of the number of words in the 4 options of the answer to the question (answer1,2,3,4)

We will use the correct column data as a label.

We need to complete function6 so that it constructs sklearn.pipeline.Pipeline in the following steps:
* sklearn.preprocessing.OrdinalEncoder applied to the gender column
* sklearn.preprocessing.FunctionTransformer applied to the question column with a function that will return the number of words ignoring characters other than letters [[hint]](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text .CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)
* sklearn.preprocessing.FunctionTransformer applied to columns answer1,2,3,4 with a function that will return the sum of the number of words in the 4 options (count as in the previous point)
* then apply sklearn.preprocessing.StandardScaler to all columns of the input data (X).
* and use sklearn.neighbors:KNeighborsClassifier with k=7 as classifier



**Download data** 🔽

In [ ]:
!wget https://raw.githubusercontent.com/NshanPotikyan/Dasa1Doom/master/files/hw4_bonus.zip
!unzip hw4_bonus.zip

--2023-05-11 20:35:07--  https://raw.githubusercontent.com/NshanPotikyan/Dasa1Doom/master/files/hw4_bonus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34872 (34K) [application/zip]
Saving to: ‘hw4_bonus.zip’

hw4_bonus.zip       100%[===================>]  34.05K  --.-KB/s    in 0.001s  

2023-05-11 20:35:07 (57.9 MB/s) - ‘hw4_bonus.zip’ saved [34872/34872]

Archive:  hw4_bonus.zip
  inflating: hw4_bonus_train.csv     
  inflating: hw4_bonus_test.csv      


In [ ]:
def function(train, test):
  X_train = train.drop(['correct'], axis=1)
  y_train = train['correct']

  X_test = test.drop(['correct'], axis=1)
  y_test = test['correct']

  class CountVectorizerSumTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      self.cv = CountVectorizer()

    def fit(self, X, y=None):
      self.cv.fit(X)
      return self

    def transform(self, X, y=None):
      v = self.cv.transform(X)
      return np.sum(v.toarray(), axis=1).reshape(-1, 1)

    def fit_transform(self, X, y=None):
      self.fit(X, y)
      return self.transform(X, y)

  class WordsCountTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      self.cv = CountVectorizer()

    def fit(self, X, y=None):
      X_concat = X.apply(lambda row: ' '.join(row), axis=1)
      self.cv.fit(X_concat)
      return self

    def transform(self, X, y=None):
      X_concat = X.apply(lambda row: ' '.join(row), axis=1)
      X_count = self.cv.transform(X_concat)
      return np.asarray(np.sum(X_count, axis=1)).reshape(-1,1)

    def fit_transform(self, X, y=None):
      self.fit(X, y)
      return self.transform(X, y)


  col_trans = ColumnTransformer(
      transformers=[
          ('gender_ordinal_encoder',OrdinalEncoder(), ['gender']),
          ('questions_word_count',CountVectorizerSumTransformer(), 'question'),
          ('answers_word_count', WordsCountTransformer(), ['answer1', 'answer2',
                                                         'answer3', 'answer4'])],
          remainder='passthrough')


  pipeline = Pipeline(
      steps=[
          ('col_trans', col_trans),
          ('scaler', StandardScaler()),
          ('classifier', KNeighborsClassifier(n_neighbors=7))
          ])
  pipeline.fit(X_train, y_train)
  accuracy_score = pipeline.score(X_test, y_test)
  return pipeline, accuracy_score


In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, TransformerMixin

train = pd.read_csv('/content/hw4_bonus_train.csv')
test = pd.read_csv('/content/hw4_bonus_test.csv')

pipeline, score = function(train, test)
print(score)
pipeline

Pipeline(steps=[('col_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('gender_ordinal_encoder',
                                                  OrdinalEncoder(),
                                                  ['gender']),
                                                 ('questions_word_count',
                                                  CountVectorizerSumTransformer(),
                                                  'question'),
                                                 ('answers_word_count',
                                                  WordsCountTransformer(),
                                                  ['answer1', 'answer2',
                                                   'answer3', 'answer4'])])),
                ('scaler', StandardScaler()),
                ('classifier', KNeighborsClassifier(n_neighbors=7))])